# Mexca Demo

**Requirements**: mexca base package, Docker

This is a short demo to illustrate how mexca can be used to extract emotion expression features from a video file. As an example, we will use a video of the presidential debate betweeen Clinton and Trump in 2016. The video can be found on [YouTube](https://www.youtube.com/watch?v=DBhrSdjePkk), but we will use a file that is hosted by a third party.

In [1]:
import logging
import os
import yaml
import pandas as pd
from huggingface_hub import notebook_login
from IPython.display import Video
from urllib.request import urlopen
from mexca.container import AudioTranscriberContainer, FaceExtractorContainer, SentimentExtractorContainer, SpeakerIdentifierContainer, VoiceExtractorContainer
from mexca.pipeline import Pipeline

Before we can apply mexca, we need to download the example video file.

In [2]:
def download_example(url, filename):
    # Check if filename exists
    if not os.path.exists(filename):
        video = urlopen(url)

        with open(filename, 'wb') as file:
            file.write(video.read())

In [3]:
example_url = 'https://books.psychstat.org/rdata/data/debate.mp4'
filename = 'debate.mp4'

download_example(example_url, filename)

Video(filename)

To track the progress of the pipeline, we create a logger from the `logging.yml` file in this directory.

In [4]:
with open('logging.yml', 'r', encoding='utf-8') as file:
    config = yaml.safe_load(file)
    logging.config.dictConfig(config)

Now that we have the example video, we can create a mexca pipeline object from containerized components. We specify that mexca should detect two faces and speakers (Clinton and Trump).

**Note**: The first time you run the pipeline with containerzied components, the containers will be automatically downloaded which can take some time.

*Note*: mexca builds on pretrained models from the pyannote.audio package. Since release 2.1.1, downloading the pretrained models requires the user to accept two user agreements on Hugging Face hub and generate an authentication token. Therefore, to run the mexca pipeline, you must accept the user agreements on [here](https://huggingface.co/pyannote/speaker-diarization) and [here](https://huggingface.co/pyannote/segmentation). Then, generate an authentication token [here](https://huggingface.co/settings/tokens). Use this token as the value for `use_auth_token` (instead of `"HF_TOKEN"`).

In [5]:
pipeline = Pipeline(
    face_extractor=FaceExtractorContainer(num_faces=2),
    speaker_identifier=SpeakerIdentifierContainer(
        num_speakers=2,
        use_auth_token=os.environ["HF_TOKEN"]#"HF_TOKEN" # Replace this string with your token
    ),
    voice_extractor=VoiceExtractorContainer(),
    audio_transcriber=AudioTranscriberContainer(),
    sentiment_extractor=SentimentExtractorContainer()
)

Next, we can apply the mexca pipeline object to the example video file. It can take a long time to process video files. Thus, we will only process 10 seconds of the video by setting the `process_subclip` argument (seconds 19 to 29). We also specify that 5 video frames should be processed at the same time (in a batch), and only every 5th frame should be processed.

In [6]:
output = pipeline.apply(filename, frame_batch_size=5, skip_frames=5, process_subclip=(19, 29))

2023-11-15 10:37:10,320 - INFO - Starting MEXCA pipeline
2023-11-15 10:37:10,610 - INFO - Wrote audio file to debate.wav
2023-11-15 10:37:10,617 - INFO - Processing video frames
100%|██████████| 10/10 [00:28<00:00,  2.88s/it]


2023-11-15 10:37:51,790 - INFO - Identifying speakers
torchvision is not available - cannot save figures

2023-11-15 10:38:18,728 - INFO - Transcribing speech segments to text
/usr/local/lib/python3.9/site-packages/whisper/timing.py:58: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def backtrace(trace: np.ndarray):

100%|██████████| 3/3 [00:32<00:00, 10.93s/it]

2023-11-15 10:38:57,424 - INFO - Extracting sentiment from transcribed text
100%|████████

The pipeline returns a `Multimodal` object that contains the extracted emotion expression features as a `polars.LazyFrame` in the `feature` attribute.

In [7]:
output.features.collect()

filename,time,frame,face_box,face_prob,face_landmarks,face_aus,face_label,face_confidence,segment_start,segment_end,segment_speaker_label,span_start,span_end,span_text,span_confidence,span_sent_pos,span_sent_neg,span_sent_neu,pitch_f0_hz,jitter_local_rel_f0,shimmer_local_rel_f0,hnr_db,f1_freq_hz,f1_bandwidth_hz,f1_amplitude_rel_f0,f2_freq_hz,f2_bandwidth_hz,f2_amplitude_rel_f0,f3_freq_hz,f3_bandwidth_hz,f3_amplitude_rel_f0,alpha_ratio_db,hammar_index_db,spectral_slope_0_500,spectral_slope_500_1500,h1_h2_diff_db,h1_f3_diff_db,mfcc_1,mfcc_2,mfcc_3,mfcc_4,spectral_flux,rms_db
str,f64,i64,list[f64],f64,list[list[f64]],list[f64],str,f64,f64,f64,str,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f32,f64,f64,f32,f64,f64,f32,f64,f64,f32,f64,f64,f64,f64,f64,f32,f64,f64,f64,f64,f64,f64
"""debate.mp4""",19.0,475,"[153.573257, 52.999695, … 99.327591]",0.99995,"[[160.349411, 73.758377], [172.529526, 68.586174], … [175.783569, 87.511871]]","[0.275031, 0.235295, … 0.019616]","""0""",0.639248,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""debate.mp4""",19.2,480,"[152.477203, 51.660679, … 99.063484]",0.999502,"[[158.934433, 73.332603], [171.008545, 66.621361], … [176.438599, 86.17321]]","[0.260472, 0.216152, … 0.022298]","""0""",0.560269,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""debate.mp4""",19.4,485,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""debate.mp4""",19.6,490,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""debate.mp4""",19.8,495,"[154.11084, 50.174015, … 97.729965]",0.999869,"[[161.535721, 70.498871], [174.275253, 64.650093], … [178.821945, 85.012604]]","[0.269365, 0.224688, … 0.027645]","""0""",0.348363,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""debate.mp4""",20.0,500,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""debate.mp4""",20.2,505,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""debate.mp4""",20.4,510,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""debate.mp4""",20.6,515,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


The column names of the data frame tell us about the features that our pipeline extracted. We can see multiple columns with the `face_` prefix that contain facial expression features and information about the detected faces. Columns with the `segment_` prefix contain information about the speech segments (note that this can be unreliable for video segments this short). Currently, mexca only extracts the voice pitch `pitch_f0` from the audio signal. The prefix `span_` indicates columns with information about sentences of the trascribed spoken text. For further information about the output and features, see the [documentation](https://mexca.readthedocs.io/en/latest/index.html).